<a href="https://colab.research.google.com/github/jiwonny29/Becoming_RENTABLE/blob/main/BERT_Fine_tuning_for_Classification_of_Accessible_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Seattle Airbnb Review Data

## Find matched accessible keywords for each review




In [6]:
import pandas as pd

# Load the Seattle reviews dataset with specified encoding
s_reviews = pd.read_csv('/content/drive/MyDrive/seattle_reviews_upd.csv', encoding='ISO-8859-1')

# Select relevant columns: 'listing_id', 'date', 'reviewer_id', 'reviewer_name', and 'comments'
s_reviews = s_reviews[['listing_id', 'date', 'reviewer_id', 'reviewer_name', 'comments']]

# Filter out rows where 'comments' is NaN (only keep rows with non-null comments)
s_reviews = s_reviews[s_reviews['comments'].notna()]

<ipython-input-6-bc4b78e2a2c0>:4: DtypeWarning: Columns (0,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  s_reviews = pd.read_csv('/content/drive/MyDrive/seattle_reviews_upd.csv', encoding='ISO-8859-1')


In [7]:
s_reviews

,listing_id,date,reviewer_id,reviewer_name,comments
0,6606,2009-07-17,18085.0,Vivian,"The Urban Cottage is comfortable, beautiful, f..."
1,6606,2011-06-27,434031.0,Elliott,Joyce was a wonderful host and the urban cotta...
2,6606,2011-08-22,976182.0,Allegra,Beautiful cottage and warm hospitality from Jo...
3,6606,2011-08-27,997921.0,Brittney,"Joyce is a wonderful host! She is warm, helpfu..."
4,6606,2011-08-30,206901.0,Pascal,Joyce's cottage is the perfect Seattle locatio...
...,...,...,...,...,...
478583,1175400861458554618,2024-06-13,543316192.0,Chinaza,Very comfortable and affordable
478584,1176438433322548146,2024-06-13,20517737.0,Drew,"Great location, walkable to lots of local spots."
478585,1176549344264444213,2024-06-21,181147877.0,Natallia,Amazing experience with the host and the place...
478586,1176828099507994596,2024-06-23,283995214.0,Victoria,This place was a stunner! Seung was really hel...


In [8]:
import re

# List of keywords to identify accessibility-related terms
keywords = ['elevator', 'wheelchair', 'stair', 'step', 'handicapped', 'ramp', 'disabled', 'disability', 'elderly', 'accessible', 'large', 'spacious', 'big', 'tub', 'lighting', 'mobility', 'older', 'balance', 'stroke', 'grandparents', 'physically', 'senior']

# Filter reviews that contain any of the keywords (case-insensitive)
accessibility_possible_reviews = s_reviews[s_reviews['comments'].str.contains('|'.join(keywords), case=False, na=False)]

# Function to find and join keywords present in each review comment
def find_keywords(text):
    return ', '.join([kw for kw in keywords if re.search(kw, text, re.IGNORECASE)])

# Apply the function to create a new column in the DataFrame with found keywords
accessibility_possible_reviews['keywords'] = accessibility_possible_reviews['comments'].apply(find_keywords)

# Code to save the filtered reviews to a CSV file (currently commented out)
# output_file_path = '/content/seattle_accessibility_reviews.csv'
# filtered_reviews.to_csv(output_file_path, index=False)

<ipython-input-8-cb0ec22a9752>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accessibility_possible_reviews['keywords'] = accessibility_possible_reviews['comments'].apply(find_keywords)


In [9]:
accessibility_possible_reviews

,listing_id,date,reviewer_id,reviewer_name,comments,keywords
5,6606,2011-09-06,552477.0,Ivy,Joyce's cottage and home were absolutely lovel...,tub
10,6606,2012-07-11,2703414.0,Susan,Loved this cottage and beautiful yard. This i...,tub
16,6606,2013-08-25,5008872.0,Beth,I really appreciated the privacy of the wee c...,large
23,6606,2014-06-14,2897283.0,Anastasia,"Joyce lives in my dream home, complete with th...",tub
31,6606,2014-09-28,2510615.0,Christopher,The cottage was cozy and comfortable and withi...,big
...,...,...,...,...,...,...
478538,1165382606020294545,2024-06-10,25040756.0,Kim,Hosts were extremely proactive and responsive....,large
478539,1165876785874732319,2024-06-09,81177811.0,Robert,From the first time I set foot on Alki in 1990...,"stair, large"
478561,1170208940040926488,2024-06-19,565141312.0,Adam,Alex and Kaila were GREAT hosts! So friendly a...,step
478575,1172478216788336535,2024-06-16,3755635.0,Jacob,Amanda's place looked exactly as we imagined i...,spacious


# Upload Custom training data for BERT modeling

In [27]:
# Load the custom training data from a CSV file in Google Drive with specific encoding
custom_training_data = pd.read_csv('/content/drive/MyDrive/bert_custom_training_data.csv', encoding='ISO-8859-1')

# Count and display the frequency of each unique value in the 'sentiment' column
custom_training_data['sentiment'].value_counts()

,count
sentiment,
Neutral,1280
Negative,450
Positive,322
positive,1


In [28]:
# Filter and sample the required number of rows for each sentiment
positive_samples = custom_training_data[custom_training_data['sentiment'] == 'Positive'].sample(n=322, random_state=42)
negative_samples = custom_training_data[custom_training_data['sentiment'] == 'Negative'].sample(n=340, random_state=42)
neutral_samples = custom_training_data[custom_training_data['sentiment'] == 'Neutral'].sample(n=338, random_state=42)

# Concatenate the samples into a single DataFrame
custom_balanced_data = pd.concat([positive_samples, negative_samples, neutral_samples], ignore_index=True)

# Display the resulting DataFrame
custom_balanced_data

,listing_id,comments,keywords,sentiment
0,3.994421e+07,This house is extremely comfortable and beauti...,wheelchair,Positive
1,4.860920e+07,We loved this home. The home was exactly what...,elderly,Positive
2,3.382268e+07,"You are renting a floor in the house, not the ...",disability,Positive
3,1.026391e+07,We really enjoyed how the place was set up. It...,disabled,Positive
4,4.992154e+17,My wife is handicap and it was very accessible...,handicap,Positive
...,...,...,...,...
995,4.893358e+07,"Good location, nice spot for my sister, her hu...",elevator,Neutral
996,1.064886e+07,Very spacious apartment. Quick walk to the con...,"step, spacious",Neutral
997,5.689689e+17,This place is wonderful! Perfect for 5 night s...,"stair, large",Neutral
998,2.607599e+07,My team and I stayed here for our annual plann...,"accessible, spacious",Neutral


In [29]:
custom_balanced_data['sentiment'].value_counts()

,count
sentiment,
Negative,340
Neutral,338
Positive,322


# Finetune pretrained BERT model to our custom training data

In [30]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch import nn
import numpy as np

# Label encoding
label_encoder = LabelEncoder()
custom_balanced_data['Sentiment'] = label_encoder.fit_transform(custom_balanced_data['sentiment'])

# Splitting train and test data
X_train, X_test, y_train, y_test = train_test_split(custom_balanced_data['comments'], custom_balanced_data['Sentiment'], test_size=0.2, random_state=42)

# Creating datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512)

class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewsDataset(train_encodings, y_train.tolist())
test_dataset = ReviewsDataset(test_encodings, y_test.tolist())

# Set weights based on class distribution
class_counts = np.bincount(y_train)
class_weights = 1. / class_counts
sample_weights = class_weights[y_train]

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Adding weights to the model’s loss function
class WeightedBertForSequenceClassification(BertForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).float().to(input_ids.device))
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits)

model = WeightedBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Set model training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Suitable epoch count for smaller datasets
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Add evaluation metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision = np.sum((preds == p.label_ids) & (preds == 1)) / np.sum(preds == 1)
    recall = np.sum((preds == p.label_ids) & (preds == 1)) / np.sum(p.label_ids == 1)
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
    accuracy = np.mean(preds == p.label_ids)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()  # Train model

# Model evaluation
results = trainer.evaluate()
print("Evaluation Results:", results)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.985800,0.977327,0.525000,0.625000,0.373134,0.467290
2,0.378200,0.799394,0.725000,0.609524,0.955224,0.744186
3,0.361100,0.529424,0.855000,0.888889,0.835821,0.861538


Evaluation Results: {'eval_loss': 0.5294240713119507, 'eval_accuracy': 0.855, 'eval_precision': 0.8888888888888888, 'eval_recall': 0.835820895522388, 'eval_f1': 0.8615384615384615, 'eval_runtime': 2.8681, 'eval_samples_per_second': 69.732, 'eval_steps_per_second': 17.433, 'epoch': 3.0}


In [31]:
model.save_pretrained('content/drive/MyDrive/bert_custom_saved_model')
tokenizer.save_pretrained('content/drive/MyDrive/tokenizer_bert_custom_saved_model')

('content/drive/MyDrive/tokenizer_bert_custom_saved_model/tokenizer_config.json',
 'content/drive/MyDrive/tokenizer_bert_custom_saved_model/special_tokens_map.json',
 'content/drive/MyDrive/tokenizer_bert_custom_saved_model/vocab.txt',
 'content/drive/MyDrive/tokenizer_bert_custom_saved_model/added_tokens.json')